In [ ]:
from pathlib import Path
from seplanet.daily import Daily

# plotting
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [18, 10]

# make tab completion
%config Completer.use_jedi = False

# Define input parameters for initializing a class instance

The main idea behind planetpy is to group all necessary functions into a class. The class is initialised by:
- a project name that will be used for order
- project directory (where all downloaded data will go, including inventories and so on)
- area of interest (single geometry provided as a GIS file, GeoJSON FeatureCollection string, geometry dictionary string or WKT string)
- start date (start of time of interest in YYYY-MM-DD format)
- end date (end of time of interest in YYYY-MM-DD format)
- max cloud cover (we can also filter later for more restrictive conditions)
- constellations (python ist obect refering to sateliite/imagery types available from Planet)
- planet API key

No additional information needs

In [ ]:
project_name = 'field_site_skysat'  # used for order title, project folder

# project directoy within home directory
project_dir = Path.home().joinpath(project_name)

# aoi example, can be geo-files (shape, gpkg, geojson), WKT string or GeoJson Strings
aoi = {"type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [[[
              36.8411922454834,
              7.673565995637892],[
              36.84659957885742,
              7.673565995637892],[
              36.84659957885742,
              7.677734016488392],[
              36.8411922454834,
              7.677734016488392],[
              36.8411922454834,
              7.673565995637892]]]}
      }

#aoi = Path.home().joinpath('Planet/SanAntonioDePale.gpkg')

# time of interest
start_date='2020-01-01'
end_date='2021-06-28'

# cloud cover
max_cloud_cover = 0.05

# planet_key
pkey = 'planet_key'

# constellation --> always needs to be a list
constellations=['SkySatScene'] #PSScene4Band']

# Initialize Class

by initializing the class, sub directories for inventory and download data will be automatically created

In [ ]:
p = Daily(
    project_name=project_name,
    project_dir=project_dir,
    aoi=aoi, 
    start_date=start_date,
    end_date=end_date,
    planet_api_key=pkey,
    constellations=constellations,
    # optional
    max_cloud_cover=max_cloud_cover, # defaults to 95
    out_projection='EPSG:4326' # defaults to EPSG:4326 (Lat/Lon), only applies for non GEE delivery
)

# Optional: set GEE delivery

GEE delivery is automatically activated if you fill the class attribute *ee_image_collection* and *ee_cloud_project*.

In [ ]:
#p.ee_cloud_project = 'ee-username'
#p.ee_image_collection = 'planet_image_collection'
#
## check if ee is connected
#import ee
#try:
#    ee.Initialize()
#    print('EE already connected, ready to go.')
#except:
#    ee.Authenticate()    

# Data inventory

Prepare a list of scenes by creating an inventory for the above defined parameters. The scenes are stored in a geodataframe that can be accessed through the *full_inventory* class property. Being a GeoDataFrame it can be used together with Geopandas functionality, e.g. *.head(5)* to show the first 5 results only.



In [ ]:
p.create_inventory()
print('')
print(f' Found {len(p.full_inventory)} scenes.')
print('')
p.full_inventory

# Plot inventory

The full inventory can be plotted like this.

In [ ]:
p.plot_inventory(p.full_inventory, transparency=0.1)

# Refine inventory

The class comes with a function *refine_inventory()* with certain options to select certain scenes. Running this function will populate the *refined_inventory* property that holds a geodataframe with the refined inventory.

In [ ]:
p.refine_inventory(
    max_cloud_cover=100, 
    scene_overlap=0, # percentage of scene overlapping with aoi (should be high for large AOIs)
    aoi_overlap=0, # percentage aoi overlapping with the scene (should be high for small AOIs)
    score=0, ## max score is 39, maybe something between 1-100 would be more intuitive?
    every='1D' # this comes from Pandas Grouper. possible values: 2D --> every 2 days; W --> Week; MS --> Month Start, YS --> year start
    # full list of options: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
)

p.refined_inventory

In [ ]:
p.plot_inventory(p.refined_inventory, transparency=0.2)

## Add pre-processing

The Planet API allows to create some basic pre-processing steps as can be found here:
https://developers.planet.com/docs/orders/tools-reference/

**NOTE:** Apart from the clipping tool, none of the others are allowed or EE delivery

In [ ]:
# clip to aoi
p.tools = ['clip']

### ATTENTION ###
# THOSE TOOLS DO NOT WORK FOR GEE delivery

# add NDVI band to reflectance image
p.tools = ['add_NDVI']

# get NDVI only (+ udm and meta)
p.tools = ['NDVI_only']

# co-register items to best selected image for ts analysis (
# NOTE: this will subset your inventory to images that overlap the reference image by at least 50%)
#p.tools = ['co-register']

# combine processing options 
# (note that some are exclusive, 
# --> e.g. co-registration and composite cannot be selected at the same time
p.tools = ['add_NDVI', 'clip']


### NOT IMPLEMETED YET ####
#------------------------------------------------

# compose images from same platform/date
# --> this will result in one order per composite
#p.tools = ['composite']


#------------------------------------------------



# Order and download

In [ ]:
# submit order
p.create_order(p.refined_inventory, resubmit=True, ask=True)

# print order request json
print('')
p.order_request

In [ ]:
# download, or check for order status (EE)
if not p.ee_cloud_project and not p.ee_image_collection:
    print('Prepapring download...')
    p.download_order()
else:
    p.get_order_status(every_seconds=15)

In [ ]:
p.get_order_status(every_seconds=15)